In [22]:
import pandas as pd
from config import weather_api_key
import requests
from pprint import pprint
from json.decoder import JSONDecodeError
import time

In [23]:
# Reading 2015 csv file
source_df = pd.read_csv('Airport_Data.csv')
source_df

,Date,Airport,Latitude,Longitude
0,2015-01-01,DFW,32.896,-97.037
1,2015-01-02,DFW,32.896,-97.037
2,2015-01-03,DFW,32.896,-97.037
3,2015-01-04,DFW,32.896,-97.037
4,2015-01-05,DFW,32.896,-97.037
...,...,...,...,...
7300,2018-12-27,ORD,41.979,-87.904
7301,2018-12-28,ORD,41.979,-87.904
7302,2018-12-29,ORD,41.979,-87.904
7303,2018-12-30,ORD,41.979,-87.904


In [24]:
source_df["Max Temp"] = ""
source_df["Precip"]=""
source_df["Precip Type"]=""
source_df["Wind Speed"]=""
source_df

,Date,Airport,Latitude,Longitude,Max Temp,Precip,Precip Type,Wind Speed
0,2015-01-01,DFW,32.896,-97.037,,,,
1,2015-01-02,DFW,32.896,-97.037,,,,
2,2015-01-03,DFW,32.896,-97.037,,,,
3,2015-01-04,DFW,32.896,-97.037,,,,
4,2015-01-05,DFW,32.896,-97.037,,,,
...,...,...,...,...,...,...,...,...
7300,2018-12-27,ORD,41.979,-87.904,,,,
7301,2018-12-28,ORD,41.979,-87.904,,,,
7302,2018-12-29,ORD,41.979,-87.904,,,,
7303,2018-12-30,ORD,41.979,-87.904,,,,


In [25]:
# loop through the sample dataframe
for index,row in source_df.iterrows():
    
    #----------Origin latitude and Longitude
    lat=row["Latitude"]
    lng=row["Longitude"]
    datetime= row["Date"]
#     https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/38.9697,-77.385/2020-10-01
#         ?key="apikey"
#         &include=obs%2Cfcst%2Cstats%2Calerts%2Ccurrent%2Chistfcst&elements=tempmax,precip,preciptype,windspeed
    # Build URL
    base_url ="https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
    query_1 = (f"{lat},{lng}/{datetime}/?key={weather_api_key}")
    query_2 = "&include=obs%2Cfcst%2Cstats%2Calerts%2Ccurrent%2Chistfcst"
    query_3 = "&elements=tempmax,precip,preciptype,windspeed"
    new_url= base_url + query_1 + query_2 + query_3
#     print(new_url)
#     get the response
    
    # Use try and except to skip the missing data
    try:
        response = requests.get(new_url).json()
        source_df.loc[index,"Max Temp"]=response['days'][0]['tempmax'] 
        source_df.loc[index,"Precip"]=response['days'][0]['precip'] 
        precip_type_list = response['days'][0]['preciptype'] 
        precip_type_str=""
        if precip_type_list != None :
            for precip_type in precip_type_list:
                precip_type_str += precip_type+","
        else:
            precip_type_str = "NA"
        source_df.loc[index,"Precip Type"]=precip_type_str 
        source_df.loc[index,"Wind Speed"]=response['days'][0]['windspeed'] 
        
    except (KeyError, IndexError, JSONDecodeError):
        print("Data not found... skipping.")
    except requests.Timeout:
        print("Request Timeout...")
    except requests.ConnectionError:
        print("ConnectionError...")
    
#     time.sleep(0.1)

# # #     pprint(response)


In [31]:
source_df.head(20)

,Date,Airport,Latitude,Longitude,Max Temp,Precip,Precip Type,Wind Speed
0,2015-01-01,DFW,32.896,-97.037,36,0.58,"rain,snow,",8
1,2015-01-02,DFW,32.896,-97.037,39.8,0.28,"rain,",11.4
2,2015-01-03,DFW,32.896,-97.037,54,0.43,"rain,",12.7
3,2015-01-04,DFW,32.896,-97.037,43.1,0,NA,27.7
4,2015-01-05,DFW,32.896,-97.037,46.1,0,NA,16.1
5,2015-01-06,DFW,32.896,-97.037,56.9,0,NA,18.3
6,2015-01-07,DFW,32.896,-97.037,39.8,0,NA,31.1
7,2015-01-08,DFW,32.896,-97.037,37.1,0,NA,18.3
8,2015-01-09,DFW,32.896,-97.037,38,0,NA,18.3
9,2015-01-10,DFW,32.896,-97.037,37.1,0.04,"rain,snow,",13.8


In [33]:
source_df.sample(n=20)

,Date,Airport,Latitude,Longitude,Max Temp,Precip,Precip Type,Wind Speed
5967,2015-05-04,ORD,41.979,-87.904,72,0,NA,17.2
901,2017-06-20,DFW,32.896,-97.037,93.8,0,NA,11.4
4031,2018-01-14,DEN,39.858,-104.667,55.1,0,NA,16.1
5160,2017-02-16,LAX,33.942,-118.408,61.9,0,NA,14.7
6234,2016-01-26,ORD,41.979,-87.904,37.1,0.27,"rain,snow,",25.3
6252,2016-02-13,ORD,41.979,-87.904,17.1,0,NA,18.3
18,2015-01-19,DFW,32.896,-97.037,69.2,0,NA,12.7
6256,2016-02-17,ORD,41.979,-87.904,34.1,0,NA,10.3
417,2016-02-22,DFW,32.896,-97.037,61.1,0.33,"rain,",13.8
6776,2017-07-21,ORD,41.979,-87.904,87.2,1.17,"rain,",13.9


In [32]:
source_df.to_csv('airport_weather_data_result.csv')